<a href="https://colab.research.google.com/github/ManishManda/HillClimbingAlgorithm/blob/main/HilClimbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def successorFunction(G,source_node,target_node):
  successor_nodes=[]
  for path in nx.all_simple_edge_paths(G, source=source_node, target=target_node, cutoff=9):
    bool=True
    for x in path:
      if G.get_edge_data(x[0],x[1])['capacity']==G.get_edge_data(x[0],x[1])['flow']:
        bool=False
        break
    if bool:
      successor_nodes.append(path)
  return successor_nodes


def heuristicValue(G,path):
  diff_edges_value=[]
  for x in path:
    diff=G.get_edge_data(x[0],x[1])['capacity']- G.get_edge_data(x[0],x[1])['flow']
    diff_edges_value.append(diff)
  diff_edges_value.sort()
  return diff_edges_value[0]

In [ ]:
import networkx as nx
from networkx.algorithms.flow import edmonds_karp
import matplotlib.pyplot as plt

source_node=0
target_node=5

G = nx.DiGraph()
G.add_edges_from([(0, 1, {'flow': 2}), (0, 1,{'capacity': 6}),
 (0, 2,{'flow': 3}), (0, 2,{'capacity': 3}),
  (0,3,{'flow': 4}), (0,3,{'capacity': 5}),
   (1,2, {'flow': 1}), (1,2,{'capacity': 5}),
    (1,5, {'flow': 1}), (1,5,{'capacity': 3}),              # modified from last version: edge (1,4) replaced by edge (1,5)
     (2,4, {'flow': 2}), (2,4,{'capacity': 9}),
      (2,5, {'flow': 4}), (2,5,{'capacity': 8}),
       (3,2, {'flow': 2}), (3,2,{'capacity': 2}),
        (3,4,{'flow': 2}), (3,4,{'capacity': 3}),
         (4,5, {'flow': 4}), (4,5,{'capacity': 5})])

def hillClimbingAlgorithm(G,source_node,target_node):
  best_solution=[]
  i=0
  while True:
    i=i+1
    best_value=0
    paths=successorFunction(G,source_node,target_node)
    for path in paths:
      value=heuristicValue(G,path)
      if value > best_value:
        best_value=value
        best_solution=path
    if best_value <= 0:
            break
    for edge in G.edges(data=True):
      node1, node2, edge_data = edge
      if (node1, node2) in best_solution or (node2, node1) in best_solution:
        edge_data['flow'] += best_value

hillClimbingAlgorithm(G,source_node,target_node)
target_node = 5

# Initialize a variable to store the sum of edge data
edge_data_sum = 0

for edge in G.edges(data=True):
    node1, node2, edge_data = edge
    if node1 == target_node or node2 == target_node:
        edge_data_sum += edge_data.get('flow', 0)
print("Optimized flow into sink node---------------------- ",edge_data_sum)


Optimized flow into sink node----------------------  14


In [ ]:

#######################R2 and R3 Question ################################################################
########################### STOTCHASTIC HILL CLIMBING ###################################
from networkx.algorithms.assortativity import connectivity
import networkx as nx
import random
N=30

def createGraph(connectivity):
  N = 30

  # Calculate the probability 'p' to achieve the desired connectivity
  p = connectivity / (N - 1)

  G=nx.gnp_random_graph(N, p, directed=True)  # here we create a directed random binomial graph with N nodes and an average (expected) connectivity of N*0.6.
  nx.is_weakly_connected(G)
  for u in G.nodes():
    in_list=G.in_edges(u)
    out_list=G.out_edges(u)
    if(len(in_list)==0) and u>0: G.add_edge(u-1,u)    # modified from previous version - need to add an incoming edge from a node that does not currently have one
    if(len(out_list)==0) and u<N-1: G.add_edge(u,u+1)
  for u in G.nodes():
    adj_list=G.adj[u]
    for nbr,datdict in adj_list.copy().items():
      if (u>nbr) and G.has_edge(nbr,u):
          G.remove_edge(u,nbr)                  # modified from last version, need a check to ensure that the forward edge exists before the backward flowing edge is deleted
  for u in G.nodes():
    adj_list=G.adj[u]
    for nbr,datdict in adj_list.items():
      if (u>nbr) and G.has_edge(nbr,u) and G.has_edge(u,nbr): print("Bidirectional edge!'")

  inflow = [0 for i in range(N)]
  outflow = [0 for i in range(N)]
  outcap = [0 for i in range(N)]
  for u in G.nodes():
    e_in=G.in_edges(u)
    e_out=G.out_edges(u)
    for x,y in e_in:
      flow=random.randint(1,6)
      G.edges[x,y]['flow']=flow             # assign flow to every incoming edge
      G.edges[x,y]['capacity']=flow*1.5
      inflow[y]=inflow[y]+G.edges[x,y]['flow']
    sum_in=inflow[u]
    edge_num=0
    temp_flow=0
    for x,y in e_out:
      if (edge_num<len(e_out)-1):
        val=random.uniform(0,1)*sum_in
        temp_flow=temp_flow+val
        sum_in=inflow[x]-temp_flow
        G.edges[x,y]['flow']=val               # assign flow to every outgoing edge
        outflow[x]=outflow[x]+val
        edge_num=edge_num+1
        val_cap=val*1.5                        # modified from previous version - need to capcaity values for edges.
        G.edges[x,y]['capacity']=val_cap      # set outgoing capacities -the sum of these need to be greater than or equal to the total inflow for a any given node
        outcap[x]=outcap[x]+val_cap
      else:
        G.edges[x,y]['flow']=sum_in           # catch the last outgoing edge
        outflow[x]=outflow[x]+sum_in
        sum_cap=sum_in*1.5
        G.edges[x,y]['capacity']=sum_cap
        outcap[x]=outcap[x]+sum_cap
  return G

source_node = 0
target_node = 29
import networkx as nx
import random


def successorFunction(G,source_node,target_node):
  successor_nodes=[]
  for path in nx.all_simple_edge_paths(G, source=source_node, target=target_node, cutoff=9):
    bool=True
    for x in path:
      if G.get_edge_data(x[0],x[1])['capacity']==G.get_edge_data(x[0],x[1])['flow']:
        bool=False
        break
    if bool:
      successor_nodes.append(path)
  return successor_nodes

def heuristicValue(G, path):
    heuristic_values = [G[edge[0]][edge[1]]['capacity'] - G[edge[0]][edge[1]]['flow'] for edge in path]
    return min(heuristic_values)

def computeProbability(dict_heuristics):
    sum_total = sum(dict_heuristics.values())
    probabilities = [h / sum_total for h in dict_heuristics.values()]
    cumulative_probabilities = [sum(probabilities[:i + 1]) for i in range(len(dict_heuristics))]
    return cumulative_probabilities

def selectChildBasedOnProbability(probabilities):
    random_number = random.random()
    for i in range(len(probabilities)):
        if random_number <= probabilities[i]:
            return i
    return len(probabilities) - 1

def hillClimbingAlgorithm(G, source_node, target_node, max_iterations=100):
    for _ in range(max_iterations):
        successor_nodes = successorFunction(G, source_node, target_node)
        dict_values = {index: heuristicValue(G, value) for index, value in enumerate(successor_nodes)}
        cumulative_probabilities = computeProbability(dict_values)
        chosen_index = selectChildBasedOnProbability(cumulative_probabilities)
        if(chosen_index>=0):
          if dict_values[chosen_index] <= 1:
            break
          chosen_path = successor_nodes[chosen_index]
          for edge in chosen_path:
            G[edge[0]][edge[1]]['flow'] += dict_values[chosen_index]

##################R2 Question ####################################
R2Graph=createGraph(3)
R = edmonds_karp(R2Graph, 0, 29)
flow_value = nx.maximum_flow_value(R, 0, 29) # The Edmonds Karp algorithm computes out the optimal flow for any given graph. Note that node 4 represents the sink node T.
print("R2--------expected flow_value",flow_value)


hillClimbingAlgorithm(R2Graph, source_node, target_node)
edge_data_sum = 0
for edge in R2Graph.edges(data=True):
    node1, node2, edge_data = edge
    if node1 == target_node or node2 == target_node:
        edge_data_sum += edge_data.get('flow', 0)
print("R2------optimized Actual  flow _value",edge_data_sum)

####################END OF R2  Question #############################################


##################  R3 Question  ########################################
for j in  range(2):
  TOTAL_FLOW_ACTUAL=0
  TOTAL_FLOW_EXPECTED=0
  for i in range(30):
    connectivity=2
    if(j==0):
      connectivity=3
    t=createGraph(connectivity)
    R = edmonds_karp(t, 0, 29)
    flow_value = nx.maximum_flow_value(R, 0, 29)
    TOTAL_FLOW_EXPECTED+=flow_value
    hillClimbingAlgorithm(t,source_node,target_node)
    target_node = 29
    # Initialize a variable to store the sum of edge data
    edge_data_sum = 0
    for edge in t.edges(data=True):
        node1, node2, edge_data = edge
        if node1 == target_node or node2 == target_node:
            edge_data_sum += edge_data.get('flow', 0)
    TOTAL_FLOW_ACTUAL+=edge_data_sum
  print("expected with connectivity ",connectivity,TOTAL_FLOW_EXPECTED/30)
  print("Actual with connectivity ",connectivity,TOTAL_FLOW_ACTUAL/30)




R2--------expected flow_value 9.0
R2------optimized Actual  flow _value 7.000000000000004
expected with connectivity 3 9.379882769685558
Actual with connectivity 3 7.567886879000
expected with connectivity 2 6.014721415792058
Actual with connectivity 2 5.170870189459974


In [ ]:
!jupyter nbconvert --to pdf /content/AI_Project_2_part_B.ipynb


[NbConvertApp] Converting notebook /content/AI_Project_2_part_B.ipynb to pdf
[NbConvertApp] Writing 51037 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 45292 bytes to /content/AI_Project_2_part_B.pdf
